In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import importlib
import argparse
import time

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.autograd as autograd
from torch.autograd import Variable
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision import transforms
import torchvision.utils as vutils
from torchnet.meter import AverageValueMeter

# Set parameters

In [3]:
parser = {
    'dataset': 'cifar10',
    'dataroot': './data',
    'workers': 2,
    'batchSize': 64,
    'imageSize': 64,
    'image_channels': 3,
    'nz': 100, #dim of latent variables
    'ngf': 64, #number of G features
    'ndf': 64, #number of D features
    'epochs': 25, 
    'lr': 1e-3,
    'beta1': 0.5, #for adam opt
    'netG': '',
    'netD': '',
    'outf': './output',
#    'ngpu': 0,
    'manualSeed': 7,
    'no_cuda': True,
}
args = argparse.Namespace(**parser)

In [4]:
print(args)

Namespace(batchSize=64, beta1=0.5, dataroot='./data', dataset='cifar10', epochs=25, imageSize=64, image_channels=3, lr=0.001, manualSeed=7, ndf=64, netD='', netG='', ngf=64, no_cuda=True, nz=100, outf='./output', workers=2)


# Load data

In [6]:
dataset = dset.CIFAR10(root=args.dataroot, download=True, transform=transforms.Compose([
                                                        transforms.Scale(args.imageSize),
                                                        transforms.ToTensor(),
                                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

assert dataset

Files already downloaded and verified


In [7]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=args.batchSize, shuffle=True, num_workers=int(args.workers))

# Define class

In [8]:
# custom weights initialization called on netG and netD
# m: layer of model
def weights_init(m):
    classname = m.__class__.__name__  #returns the name of class of m
    if classname.find('Conv') != -1:  #name contains Conv
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1: #name contains BatchNorm, this can be seen like activation function after batchnorm
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

```class torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True)```

```python
def __init__(self):
        self._backend = thnn_backend
        self._parameters = OrderedDict()
        self._buffers = OrderedDict()
        self._backward_hooks = OrderedDict()
        self._forward_hooks = OrderedDict()
        self._modules = OrderedDict()
        self.training = True
```

```class torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, groups=1, bias=True)```

In [32]:
#Define class Generator:
class _netG(nn.Module):
    #def __init__(self, ngpu):
    def __init__(self):
        super(_netG, self).__init__()  #no need to list __init__ of nn.Module
        #self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(args.nz, args.ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(args.ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(args.ngf * 8, args.ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(args.ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(args.ngf * 4, args.ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(args.ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(args.ngf * 2, args.ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(args.ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(args.ngf, args.image_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64 --> image size = 64
        )

    def forward(self, input):
        #gpu_ids = None
        #if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            #gpu_ids = range(self.ngpu)
        #return nn.parallel.data_parallel(self.main, input, gpu_ids)
        return self.main(input)

In [33]:
netG = _netG()
netG.apply(weights_init)
print(netG)

_netG (
  (main): Sequential (
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU (inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh ()
  )
)


```class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)```

In [34]:
class _netD(nn.Module):
    def __init__(self):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(args.image_channels, args.ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(args.ndf, args.ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(args.ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(args.ndf * 2, args.ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(args.ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(args.ndf * 4, args.ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(args.ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(args.ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        #output = nn.parallel.data_parallel(self.main, input, gpu_ids)
        output = self.main(input)
        return output.view(-1, 1)

In [35]:
netD = _netD()
netD.apply(weights_init)
print(netD)

_netD (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU (0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid ()
  )
)


# Train

In [36]:
criterion = nn.BCELoss()

```class torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)```

In [37]:
optimizerD = optim.Adam(netD.parameters(), lr=args.lr, betas=(args.beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=args.lr, betas=(args.beta1, 0.999))

**NOTE**: 
if update Pytorch, the steps when training will be changed:

buffer --> backward --> update _**CHANGED TO**_ backward --> buffer --> update

In [41]:
def train(epoch):
    D_losses = AverageValueMeter()
    G_losses = AverageValueMeter()
    D_real_accuracies = AverageValueMeter()
    D_fake_accuracies = AverageValueMeter()
    G_real_accuracies = AverageValueMeter()
   
    start = time.time()
    for i, (real, _) in enumerate(dataloader):
        
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        batch_size = real.size(0)
        
        # Create ones_label and zeros_label
        real_label = Variable(torch.ones(batch_size))
        fake_label = Variable(torch.zeros(batch_size))
        z = Variable(torch.randn(batch_size, args.nz, 1,1))
        
        #train with real
        real = Variable(real)
        real_output = netD(real)
        D_real_loss = criterion(real_output, real_label)
        
        netD.zero_grad()
        D_real_loss.backward()
        
        
        D_real_accuracy = real_output.data.mean()
        
        #train with fake
        fake = netG(z)
        fake_output = netD(fake.detach())   #Use detach so we just need to creat z once
        D_fake_loss = criterion(fake_output, fake_label)
        
        D_fake_loss.backward()
        D_fake_accuracy = fake_output.data.mean()
        
        
        D_loss = D_real_loss + D_fake_loss
        optimizerD.step()
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        
        output = netD(fake)
        G_loss = criterion(output, real_label)
        
        netG.zero_grad()
        G_loss.backward()
        
        G_real_accuracy = output.data.mean()
        
        optimizerG.step()
        
        D_losses.add(D_loss.data[0] * batch_size, batch_size)
        G_losses.add(G_loss.data[0] * batch_size, batch_size)
        D_real_accuracies.add(D_real_accuracy * batch_size, batch_size)
        D_fake_accuracies.add(D_fake_accuracy * batch_size, batch_size)
        G_real_accuracies.add(G_real_accuracy * batch_size, batch_size)
        
        print("=> EPOCH {:2d} | Time: {}s | D_loss: {:.3f} | G_loss: {:.3f} "
          "| D_real_acc: {:.3f} | D_fake_acc: {:.3f} | G_real_acc: {:.3f}"
          .format(epoch, int(time.time()-start), D_losses.value()[0],
                  G_losses.value()[0], D_real_accuracies.value()[0],
                  D_fake_accuracies.value()[0], G_real_accuracies.value()[0]))
    
        
        if i % 100 == 0:
            plot(real, epoch)

In [44]:
def plot(X, epoch):
    z = Variable(torch.randn(X.size(0), args.nz, 1,1))
    vutils.save_image(X.data, '%s/real_samples.png' % args.outf)
    fake = netG(z)
    vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03d.png' % (args.outf, epoch))

In [45]:
for epoch in range(1, 6):
    train(epoch)

=> EPOCH  1 | Time: 4s | D_loss: 2.643 | G_loss: 27.404 | D_real_acc: 0.515 | D_fake_acc: 0.564 | G_real_acc: 0.000
=> EPOCH  1 | Time: 12s | D_loss: 2.600 | G_loss: 25.877 | D_real_acc: 0.454 | D_fake_acc: 0.282 | G_real_acc: 0.000
=> EPOCH  1 | Time: 17s | D_loss: 8.692 | G_loss: 24.875 | D_real_acc: 0.615 | D_fake_acc: 0.521 | G_real_acc: 0.000
=> EPOCH  1 | Time: 22s | D_loss: 7.191 | G_loss: 25.449 | D_real_acc: 0.660 | D_fake_acc: 0.554 | G_real_acc: 0.000
=> EPOCH  1 | Time: 26s | D_loss: 6.110 | G_loss: 25.781 | D_real_acc: 0.644 | D_fake_acc: 0.444 | G_real_acc: 0.000
=> EPOCH  1 | Time: 31s | D_loss: 5.167 | G_loss: 25.208 | D_real_acc: 0.665 | D_fake_acc: 0.370 | G_real_acc: 0.000
=> EPOCH  1 | Time: 35s | D_loss: 7.700 | G_loss: 22.776 | D_real_acc: 0.706 | D_fake_acc: 0.460 | G_real_acc: 0.017
=> EPOCH  1 | Time: 40s | D_loss: 8.291 | G_loss: 21.899 | D_real_acc: 0.726 | D_fake_acc: 0.520 | G_real_acc: 0.017
=> EPOCH  1 | Time: 45s | D_loss: 7.695 | G_loss: 20.743 | D_real

Process Process-14:
Process Process-13:
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 26, in _worker_loop
    r = index_queue.get()
  File "/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 26, in _worker_loop


KeyboardInterrupt: 